In [43]:
import gzip
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import random
import os

In [44]:
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

## Data Loading

### Writing PyG Dataset

In [45]:
import torch

In [46]:
from torch_geometric.data import InMemoryDataset
from torch.utils.data import DataLoader

In [47]:
from torch_geometric.utils import from_networkx, to_networkx

In [48]:
def clustering_coefficient(G, node):
    ns = [n for n in G.neighbors(node)]
    if len(ns) <= 1:
        return 0
    
    numerator = 0
    denominator = len(ns) * (len(ns) - 1) / 2
    for i in range(0, len(ns)):
        for j in range(i+1, len(ns)):
            n1, n2 = ns[i], ns[j]
            numerator += G.has_edge(n1, n2)
    
    return numerator / denominator
                

In [49]:
def generate_feature_vector(G):
    x = torch.randn(G.number_of_nodes(), 4)
    ind = 0
    for node in G.nodes():
        x[ind][0] = 1 # uniform
        x[ind][1] = G.degree[node] # node degree as a scalar 
        x[ind][2] = clustering_coefficient(G, node) # triangle counting?
        x[ind][3] = ind # node ID features
        ind += 1
    return x

In [50]:
class LamanDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data.pt']
        
    def process(self):
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        for ind, graph in enumerate(total_laman_data[0]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        for ind, graph in enumerate(total_laman_data[1]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [51]:
DATA_PATH = "data/custom-generated.pkl.gz"

In [52]:
laman_data = LamanDataset("", DATA_PATH)

Processing...
Done!


In [53]:
laman_data[0]

Data(edge_index=[2, 58], x=[15, 4], label=[1], num_nodes=15)

## Split into Train / Test

In [57]:
from torch.utils.data import random_split

proportions = [.7, .3]
lengths = [int(p * len(laman_data)) for p in proportions]
lengths[-1] = len(laman_data) - sum(lengths[:-1])

generator1 = torch.Generator().manual_seed(42)
train_data, test_data = random_split(laman_data, lengths, generator=generator1)

In [58]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size = 64, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 64, shuffle=True)

In [59]:
print("Number of train batches: ", len(train_loader))
print("Number of test batches: ", len(test_loader))

Number of train batches:  975
Number of test batches:  418


In [60]:
for data in train_loader:
    print(data)
    break

DataBatch(edge_index=[2, 3870], x=[960, 4], label=[64], num_nodes=960, batch=[960], ptr=[65])


## Model Architecture & Training

In [86]:
from gin.gin_k_layers import GIN

In [87]:
model = GIN(layers = 2, num_features=1, dim_h=20)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GIN(
  (conv_layers): ModuleList(
    (0): GINConv(nn=Sequential(
      (0): Linear(in_features=1, out_features=20, bias=True)
      (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=20, out_features=20, bias=True)
      (4): ReLU()
    ))
    (1): GINConv(nn=Sequential(
      (0): Linear(in_features=20, out_features=20, bias=True)
      (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=20, out_features=20, bias=True)
      (4): ReLU()
    ))
  )
  (lin1): Linear(in_features=120, out_features=120, bias=True)
  (lin2): Linear(in_features=120, out_features=1, bias=True)
)
Number of parameters:  16021


In [88]:
from torch_geometric.data import DataLoader
from torch.nn import BCELoss
import warnings
warnings.filterwarnings("ignore")

loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.001)

# scheduler = ReduceLROnPlateau(optimizer, 'min', min_lr=1e-6, verbose=True, patience=10)

In [89]:
def train(model, data, features_to_use):
    ind = 0
    for batch in data:
        optimizer.zero_grad()
        pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
        pred = torch.squeeze(pred)
        loss = loss_fn(pred.float(), batch.label.float())
        loss.backward()
        optimizer.step()
        ind += 1

    return loss, model

In [90]:
def check_accuracy(model, loader, features_to_use):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            y = batch.label
            batch.label = 0
            pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
            pred = torch.squeeze(pred)
            predictions = (pred > 0.5).long() 
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
    return float(num_correct)/float(num_samples)*100

In [91]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(600):
    loss, model = train(model, train_loader, [3])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model, train_loader, [3]), check_accuracy(model, test_loader, [3])
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

Starting training...
Epoch 0 | Train loss 0.4517399072647095
Train Accuracy 76.58503750240399 | Test Accuracy 76.94637648642585
Epoch 1 | Train loss 0.2509089410305023
Train Accuracy 84.67529969869864 | Test Accuracy 84.73562186822228
Epoch 2 | Train loss 0.29067057371139526
Train Accuracy 87.41265465734983 | Test Accuracy 87.39062149427866
Epoch 3 | Train loss 0.2411557137966156
Train Accuracy 88.02968138983267 | Test Accuracy 87.88048762246653
Epoch 4 | Train loss 0.19316844642162323
Train Accuracy 88.43515609974997 | Test Accuracy 88.2693889761424
Epoch 5 | Train loss 0.2930819094181061
Train Accuracy 88.59542278351176 | Test Accuracy 88.25069179567721
Epoch 6 | Train loss 0.2088492065668106
Train Accuracy 87.428681325726 | Test Accuracy 87.2634806671154
Epoch 7 | Train loss 0.23042112588882446
Train Accuracy 88.99128149240336 | Test Accuracy 88.74055792386508


KeyboardInterrupt: 

In [ ]:
# generate a twenty node graph

In [ ]:
# generate fully connected 11-node graph

In [ ]:
# generate an 20-node graph
n = 30

In [ ]:
g = nx.Graph()

In [ ]:
for i in range(0, 30):
    j = (np.random.randint(0, 30))
    while j == i:
        j = (np.random.randint(0, 30))
        
    print(j == i)
    g.add_edge(i, j)

In [ ]:
all_possible_edges = set()
for i in range(0, 30):
    for j in range(i+1, 30):
        if not g.has_edge(i, j):
            all_possible_edges.add((i,j))

In [ ]:
print(all_possible_edges)

In [ ]:
import random 
non_edges_sample_order = random.sample(all_possible_edges, 57 - 31)

In [ ]:
for (u,v) in non_edges_sample_order:
    g.add_edge(u, v)

In [ ]:
print(g.number_of_edges())

In [ ]:
model.eval()
with torch.no_grad():
    graph_as_data = from_networkx(g)
    graph_as_data.x = generate_feature_vector(g)
    validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)
    for batch in validation_set:
        pred = model(batch.x[:, [0]], batch.edge_index, batch.batch)
        print(pred)


In [ ]:
check_accuracy(model, test_loader, [0])

In [ ]:
check_accuracy(model, train_loader, [0, 1, 2, 3])

In [ ]:
# sanity check

In [ ]:
# test on best model

In [37]:
# square
import networkx as nx
square = nx.Graph()
square.add_edge(0, 1)
square.add_edge(1, 3)
square.add_edge(0, 2)
square.add_edge(2, 3)

In [38]:
# square with cross bar (rigid)
import networkx as nx
square_bar = nx.Graph()
square_bar.add_edge(0, 1)
square_bar.add_edge(1, 3)
square_bar.add_edge(0, 2)
square_bar.add_edge(2, 3)
square_bar.add_edge(0, 3)
square_bar.add_edge(1, 2)

In [39]:
# triangle
import networkx as nx
triangle = nx.Graph()
triangle.add_edge(0, 1)
triangle.add_edge(0, 2)
triangle.add_edge(1, 2)

In [40]:
# pentagon
import networkx as nx
pentagon = nx.Graph()
pentagon.add_edge(0, 1)
pentagon.add_edge(1, 3)
pentagon.add_edge(3, 4)
pentagon.add_edge(4, 2)
pentagon.add_edge(2, 0)

In [42]:
toy_problems = [square, square_bar, triangle, pentagon]
labels = [1, 0, 0, 1]

for index, toy_problem in enumerate(toy_problems):
    model.eval()
    with torch.no_grad():
        graph_as_data = from_networkx(toy_problem)
        graph_as_data.x = generate_feature_vector(toy_problem)
#         graph_as_data.label = labels[index]
        validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)
        for batch in validation_set:
            pred = model(batch.x[:, [0, 1, 2, 3]], batch.edge_index, batch.batch)
            print(pred)
    

(tensor([[1.]]), tensor([[43.0925]]))
(tensor([[1.]]), tensor([[62.7327]]))
(tensor([[1.]]), tensor([[103.2396]]))
(tensor([[1.]]), tensor([[66.0016]]))


In [ ]:
graph_as_data

In [ ]:
validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)

In [ ]:
for batch in validation_set:
    pred = bestModel(batch.x, batch.edge_index, batch.batch)
    print(pred[0])
    print(pred[1])

In [ ]:
class LamanTestDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_test.pt']
        
    def process(self):
        # processing code here
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        ind = 0
        # convert from graph to Data object
        for graph in total_laman_data[0]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 1)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        ind = 0
        for graph in total_laman_data[1]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 64)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
# add functionality to support a test dataset
TEST_DATA_PATH = "../data-2d/data/test-dataset-30loc-5std.pkl.gz"
laman_test_set = LamanTestDataset("", TEST_DATA_PATH)

In [ ]:
from torch_geometric.loader import DataLoader
laman_test_loader = DataLoader(laman_test_set, batch_size = 2, shuffle=True)

In [ ]:
random_test_acc = check_accuracy(bestModel, laman_test_loader)
print(f"Accuracy {random_test_acc}")

In [ ]:
# generate statistics on the data

In [ ]:
# test the clustering coefficient

In [ ]:
clustering_coefficient(square, 0)

In [ ]:
clustering_coefficient(triangle, 0)

In [ ]:
clustering_coefficient(square_bar, 0)

In [ ]:
# generate graph correlating clustering coefficient to rigidity

In [ ]:
for item in train_data:
    item = to_networkx(item)
    print(type(item))
    
    break

In [ ]:
torch_geometric.utils.convert.to_networkx()

# Scratch Work: Sahil

In [ ]:
total_laman_data = None
with gzip.open(DATA_PATH, 'r') as f:
    total_laman_data = pickle.load(f)

In [ ]:
sample_graph = total_laman_data[0][0]

In [ ]:
print(type(sample_graph))

In [ ]:
to_data = from_networkx(sample_graph)
from_data = to_networkx(to_data, to_undirected = True)

In [ ]:
print(type(from_data))

In [ ]:
def compute_min_clustering_coefficient(G):
    min_coefficient = 1
    for node in G.nodes():
        min_coefficient = min(min_coefficient, clustering_coefficient(G, node))
        
    return min_coefficient

In [ ]:
for index, sample_graph in enumerate(train_data):
    label = sample_graph.label
    networkx_sample_graph = to_networkx(sample_graph, to_undirected = True)
    print(label, " ", index, " ", compute_min_clustering_coefficient(networkx_sample_graph))
    
    if index == 10:
        break

In [ ]:
print(compute_min_clustering_coefficient(from_data))

In [ ]:
compute_min_clustering_coefficient(triangle)

In [ ]:
# what if instead of training a gnn – you just trained on 

In [ ]:
# train a binary classifier on just the degrees of the nodes



In [ ]:
# train a binary classifier on just the triangle feature

In [ ]:
# train a binary classifier on the triangle features and the degree

In [ ]:
# train a network with just degree of the node 

In [ ]:
model_just_degree = GIN(num_features=1)
print(model_just_degree)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

In [ ]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(1000):
    loss, h = train(train_loader, [0])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model_just_degree, train_loader), check_accuracy(model_just_degree, test_loader)
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
#     scheduler.step(test_acc)
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

In [ ]:
generate_feature_vector(sample_graph)

In [ ]:
sample_graph.x[:, [0, 1]]